In [1]:
# Exports QuantumCollocation, NamedTrajectories, and TrajectoryIndexingUtils
using Piccolo
using LinearAlgebra

# Plots
using CairoMakie

In [2]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units

const n_qubits = 1
const n_levels = 2

t_f = 10 * ns
n_steps = 101
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]

0.1

In [3]:
# Operators
Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
Rx(theta)  = kron(Paulis["I"],exp(-im * theta * Paulis["X"]/2))
Ry(theta)  = kron(Paulis["I"],exp(-im * theta * Paulis["Y"]/2))
Rz(theta)  =  kron(Paulis["I"],exp(-im * theta * Paulis["Z"]/2));

In [4]:
a = [0 1; 0 0]
ad = transpose(a)
a0  = kron(a,Paulis["I"])
ad0  = kron(ad,Paulis["I"])
a1  = kron(Paulis["I"],a)
ad1  = kron(Paulis["I"],ad);

In [5]:
H_drift = [ ]
H_drives = [
    ad0 * a0,
    ad1 * a1,
    a0+ad0,
    im * (a0-ad0),
    a1+ad1,
    im * (a1-ad1),
    ad0 * a1 + a0 * ad1
]
system = QuantumSystem(H_drives);

In [6]:
PICO_max_iter = 100

# Shape the cost function with weights on states and controls
Q = 100.
R = 1e-4

# Add control bounds
a_bound = 2 * π * 100 * MHz
dda_bound = .02
initial = nothing
infidelities = []
trajectories = []
for i in 0:20
    target = Rx(i*pi/10)
    println("Angle "*string(i*pi/10))
    println("-----------------------")
    p = UnitarySmoothPulseProblem(
        system,
        target,
        n_steps,
        Δt;
        a_bound=a_bound,
        dda_bound=dda_bound,
        Q=Q,
        R=R,
        verbose=true,
        hessian_approximation=true,
        pade_order=10,
        free_time=true,
        timesteps_all_equal=false,
        subspace=[1,2,3,4],
        max_iter=PICO_max_iter,
        #init_trajectory=initial
    )
    solve!(p)

    result = copy(p.trajectory)
    initial = result
    push!(trajectories,result)
    states = map(iso_vec_to_operator, eachslice(result[:Ũ⃗], dims=2))
    println("Infidelity " *string(unitary_infidelity(result[:Ũ⃗][:, end], operator_to_iso_vec(target))))
    rollout_states = unitary_rollout(result, system; integrator=exp)
    ΔUs = map(norm, eachslice(rollout_states .- result[:Ũ⃗], dims=2))
    println("Rollout Error "*string(maximum(ΔUs)))
    println("Infidelity "*string(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target))))
    push!(infidelities,unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target)))

end

Angle 0.0
-----------------------
    building dynamics from integrators...
        constructing knot point dynamics functions...
        determining dynamics derivative structure...
            computing jacobian sparsity...
            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
************************************************************************

  71  3.9183374e-06 3.68e-08 4.00e+01  -4.1 2.24e-03    -  1.00e+00 2.50e-01h  3
  72  6.3519367e-06 5.35e-08 2.07e+01  -4.1 9.54e-04    -  1.00e+00 5.00e-01h  2
  73  5.5990788e-06 1.30e-07 2.99e+01  -4.1 1.75e-03    -  1.00e+00 5.00e-01h  2
  74  5.4816353e-06 2.15e-06 2.91e+01  -4.1 3.46e-03    -  1.00e+00 1.00e+00h  1
  75  6.9187012e-06 2.48e-07 2.44e+01  -4.1 1.07e-03    -  1.00e+00 1.00e+00h  1
  76  2.7000592e-06 3.16e-07 2.49e+01  -4.1 1.73e-03    -  1.00e+00 1.00e+00h  1
  77  5.3608259e-05 7.69e-06 2.51e+01  -4.1 3.79e-02    -  1.00e+00 5.00e-01h  2
  78  4.0335901e-05 9.71e-07 2.52e+01  -4.1 5.12e-03    -  1.00e+00 1.00e+00h  1
  79  2.2129064e-05 1.76e-06 2.49e+01  -4.1 1.63e-02    -  1.00e+00 5.00e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  1.0354293e-05 2.86e-07 2.51e+01  -4.1 2.06e-03    -  1.00e+00 1.00e+00h  1
  81  1.1577805e-06 4.81e-07 2.48e+01  -4.1 2.34e-03    -  1.00e+00 1.00e+00h  1
  82  7.1962736e-06 1.77e-07

  32  4.9709716e-05 2.03e-06 1.60e+01  -4.0 1.66e-02    -  1.00e+00 6.25e-02h  5
  33  8.7969717e-06 1.51e-06 3.70e+01  -4.1 4.12e-03    -  1.00e+00 5.00e-01h  2
  34  3.6856198e-05 3.95e-06 4.18e+01  -4.0 1.42e-02    -  1.00e+00 2.50e-01h  3
  35  4.4834666e-05 2.73e-06 3.60e+01  -4.1 4.67e-03    -  1.00e+00 5.00e-01h  2
  36  4.5734037e-05 1.01e-05 1.12e+01  -4.1 1.09e-02    -  1.00e+00 5.00e-01h  2
  37  4.0143234e-05 5.30e-06 3.32e+01  -4.1 3.33e-03    -  1.00e+00 5.00e-01h  2
  38  1.2834672e-05 4.27e-06 4.14e+01  -4.1 5.23e-03    -  1.00e+00 5.00e-01h  2
  39  5.7768827e-05 1.24e-06 1.19e+01  -4.1 3.20e-03    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40  8.1582002e-05 1.32e-06 3.46e+01  -4.1 4.05e-03    -  1.00e+00 5.00e-01h  2
  41  4.2212182e-05 1.35e-06 1.83e+01  -4.1 1.61e-03    -  1.00e+00 1.00e+00h  1
  42  1.4406118e-05 5.37e-07 1.58e+01  -4.1 8.42e-04    -  1.00e+00 1.00e+00h  1
  43  3.3486782e-05 3.59e-07

   1  6.4891407e-01 7.72e-01 2.38e+01  -1.3 7.05e-01    -  1.61e-02 3.60e-01f  1
   2  6.2814207e-01 6.62e-01 2.38e+01  -0.5 7.35e-01    -  4.63e-01 1.43e-01f  1
   3  1.2209533e+00 9.34e-03 2.57e+01  -0.6 3.72e-01    -  9.29e-01 1.00e+00f  1
   4  1.9218476e-01 1.63e-04 2.76e+01  -1.7 9.98e-02    -  1.00e+00 1.00e+00h  1
   5  2.6902576e+00 1.46e-04 3.41e+01  -1.8 1.61e-01    -  9.67e-01 1.00e+00H  1
   6  1.1474815e+00 1.46e-04 2.76e+01  -2.4 8.13e-02    -  1.00e+00 1.00e+00f  1
   7  1.0867778e+00 1.09e-03 2.09e+01  -2.8 1.67e-01    -  1.00e+00 1.00e+00f  1
   8  2.3412545e-01 5.84e-04 2.38e+01  -2.9 1.23e-01    -  7.22e-01 1.00e+00f  1
   9  3.5528351e-01 2.79e-04 2.64e+01  -2.9 7.04e-02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.8120184e-02 1.39e-04 2.80e+01  -2.9 4.89e-02    -  1.00e+00 1.00e+00h  1
  11  1.7901601e-02 1.39e-04 2.02e+01  -2.7 5.97e-01    -  1.00e+00 3.12e-02f  6
  12  4.3051459e-03 2.32e-05

  94  1.3002397e-05 1.26e-05 2.38e+01  -4.1 1.36e-01    -  1.00e+00 1.25e-01h  4
  95  1.4369310e-05 9.41e-06 2.38e+01  -4.1 2.44e-02    -  1.00e+00 5.00e-01h  2
  96  9.3080112e-06 1.03e-05 2.38e+01  -4.0 8.32e-02    -  1.00e+00 1.25e-01h  4
  97  2.1544214e-05 9.17e-06 2.38e+01  -4.1 3.48e-02    -  1.00e+00 2.50e-01h  3
  98  6.5257268e-06 8.03e-06 2.38e+01  -4.0 3.24e-02    -  1.00e+00 2.50e-01h  3
  99  2.1667173e-05 7.11e-06 2.37e+01  -4.0 3.33e-02    -  1.00e+00 2.50e-01h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  2.0097034e-05 6.59e-06 2.37e+01  -4.0 3.14e-02    -  1.00e+00 1.25e-01h  4

Number of Iterations....: 100

                                   (scaled)                 (unscaled)
Objective...............:   2.0097034100989316e-05    2.0097034100989316e-05
Dual infeasibility......:   2.3705204133142363e+01    2.3705204133142363e+01
Constraint violation....:   6.5936843646197119e-06    6.5936843646197119e-06
Variable bound vio

...

  94  2.8987658e-04 6.25e-06 2.24e+01  -4.1 1.28e-02    -  1.00e+00 1.00e+00h  1
  95  2.8440729e-04 4.97e-06 2.24e+01  -4.1 1.48e-02    -  1.00e+00 5.00e-01h  2
  96  2.7359109e-04 6.72e-06 2.24e+01  -4.1 2.39e-02    -  1.00e+00 5.00e-01h  2
  97  2.8129135e-04 6.35e-06 2.21e+01  -4.1 2.38e-02    -  1.00e+00 5.00e-01h  2
  98  2.6488382e-04 3.56e-06 2.24e+01  -4.1 9.85e-03    -  1.00e+00 1.00e+00h  1
  99  2.7546882e-04 3.07e-06 2.22e+01  -4.1 9.07e-03    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  2.7533611e-04 2.74e-06 2.24e+01  -4.1 1.55e-02    -  1.00e+00 1.25e-01h  4

Number of Iterations....: 100

                                   (scaled)                 (unscaled)
Objective...............:   2.7533611019851703e-04    2.7533611019851703e-04
Dual infeasibility......:   2.2360713628384005e+01    2.2360713628384005e+01
Constraint violation....:   2.7358629165474824e-06    2.7358629165474824e-06
Variable bound vio

  55  3.0236152e-04 3.26e-06 2.30e+01  -4.1 1.49e-02    -  1.00e+00 5.00e-01h  2
  56  3.0122348e-04 2.30e-06 2.31e+01  -4.1 8.15e-03    -  1.00e+00 1.00e+00h  1
  57  3.0932011e-04 1.01e-06 2.34e+01  -4.1 5.51e-03    -  1.00e+00 1.00e+00h  1
  58  3.5868531e-04 9.42e-07 2.34e+01  -4.1 1.17e-02    -  1.00e+00 2.50e-01h  3
  59  3.3300297e-04 6.97e-07 2.35e+01  -4.1 1.10e-02    -  1.00e+00 2.50e-01h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  60  2.9561743e-04 1.01e-06 2.29e+01  -4.0 2.11e-02    -  1.00e+00 2.50e-01h  3
  61  2.9578642e-04 7.85e-06 2.33e+01  -4.1 1.29e-02    -  1.00e+00 1.00e+00h  1
  62  2.9555544e-04 7.71e-06 2.33e+01  -4.1 1.28e-02    -  1.00e+00 1.00e+00h  1
  63  3.0927528e-04 2.57e-06 2.31e+01  -4.1 9.86e-03    -  1.00e+00 1.00e+00h  1
  64  2.9940425e-04 1.80e-06 2.33e+01  -4.1 1.04e-02    -  1.00e+00 5.00e-01h  2
  65  2.9474680e-04 2.19e-06 2.34e+01  -4.1 1.27e-02    -  1.00e+00 5.00e-01h  2
  66  3.0555251e-04 8.59e-07

  16  1.6891996e-03 6.04e-06 1.70e+01  -4.0 4.28e-03    -  1.00e+00 1.00e+00h  1
  17  1.2746070e-03 3.04e-06 3.12e+01  -4.0 5.56e-03    -  1.00e+00 1.00e+00h  1
  18  3.9464940e-04 5.07e-06 2.55e+01  -4.0 5.41e-03    -  1.00e+00 1.00e+00h  1
  19  5.0193806e-04 5.01e-06 2.41e+01  -3.7 3.09e-02    -  1.00e+00 1.56e-02h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  4.0736533e-04 2.10e-06 1.99e+01  -3.8 4.16e-03    -  1.00e+00 1.00e+00h  1
  21  6.9862814e-04 4.95e-06 3.36e+01  -4.0 5.54e-03    -  1.00e+00 1.00e+00H  1
  22  6.7850657e-04 4.36e-06 3.04e+01  -4.1 4.51e-03    -  1.00e+00 1.00e+00h  1
  23  5.5666082e-04 2.70e-05 1.90e+01  -4.1 1.23e-02    -  9.40e-01 1.00e+00h  1
  24  3.8549483e-04 7.42e-05 2.26e+01  -4.1 1.62e-02    -  1.00e+00 1.00e+00h  1
  25  6.6303573e-04 5.48e-05 2.65e+01  -4.1 7.98e-03    -  1.00e+00 2.50e-01h  3
  26  4.7787942e-04 1.86e-06 2.70e+01  -4.1 3.14e-03    -  1.00e+00 1.00e+00h  1
  27  6.0486896e-04 2.01e-06

    building dynamics from integrators...
        constructing knot point dynamics functions...
        determining dynamics derivative structure...
            computing jacobian sparsity...
            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:    81906
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     5408
                     variables with only lower bounds:        0
                vari

  76  3.7948561e-04 4.75e-06 2.46e+01  -4.1 2.07e-02    -  1.00e+00 5.00e-01h  2
  77  3.7060076e-04 2.87e-06 2.44e+01  -4.1 1.79e-02    -  1.00e+00 5.00e-01h  2
  78  3.6807005e-04 2.50e-06 2.46e+01  -4.1 1.11e-02    -  1.00e+00 5.00e-01h  2
  79  3.6738793e-04 3.17e-06 2.44e+01  -4.1 6.23e-02    -  1.00e+00 1.25e-01h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  3.8191977e-04 1.90e-06 2.46e+01  -4.1 5.72e-03    -  1.00e+00 1.00e+00h  1
  81  3.7008134e-04 1.74e-06 2.44e+01  -4.1 7.35e-03    -  1.00e+00 5.00e-01h  2
  82  3.6837356e-04 2.64e-06 2.43e+01  -4.1 1.46e-02    -  1.00e+00 5.00e-01h  2
  83  3.7313860e-04 1.52e-06 2.47e+01  -4.1 7.60e-03    -  1.00e+00 1.00e+00h  1
  84  3.7156810e-04 1.49e-06 2.47e+01  -4.1 9.94e-03    -  1.00e+00 5.00e-01h  2
  85  3.8065581e-04 1.17e-06 2.47e+01  -4.1 2.17e-02    -  1.00e+00 2.50e-01h  3
  86  3.7059118e-04 1.39e-06 2.43e+01  -4.1 5.42e-02    -  1.00e+00 6.25e-02h  5
  87  3.7170141e-04 1.23e-06

In [7]:
infidelities

21-element Vector{Any}:
 3.220701119133196e-7
 6.721505284579266e-7
 0.00029745493142718793
 1.143237710610201e-6
 7.066696117163929e-9
 3.7317594436037105e-5
 7.693941163677831e-5
 6.4834262267776666e-6
 2.1260210901208332e-5
 0.00039214787066377266
 0.007240989823907862
 0.0002965835146594431
 0.0004736839805744575
 2.33360669976701e-6
 0.00014935253323922204
 9.874541788135183e-6
 1.4109179766963464e-5
 5.2786762820722366e-5
 0.00010125599248822503
 8.344080005828758e-7
 2.4581940210821607e-5

In [8]:
PICO_max_iter = 100

# Shape the cost function with weights on states and controls
Q = 100.
R = 1e-4

# Add control bounds
a_bound = 2 * π * 100 * MHz
dda_bound = .02
initial = nothing
initialized_infidelities = []
initialized_trajectories = []
for i in 0:20
    target = Rx(i*pi/10)
    println("Angle "*string(i*pi/10))
    println("-----------------------")
    p = UnitarySmoothPulseProblem(
        system,
        target,
        n_steps,
        Δt;
        a_bound=a_bound,
        dda_bound=dda_bound,
        Q=Q,
        R=R,
        verbose=true,
        hessian_approximation=true,
        pade_order=10,
        free_time=true,
        timesteps_all_equal=false,
        subspace=[1,2,3,4],
        max_iter=PICO_max_iter,
        init_trajectory=initial
    )
    solve!(p)

    result = copy(p.trajectory)
    initial = result
    push!(initialized_trajectories,result)
    states = map(iso_vec_to_operator, eachslice(result[:Ũ⃗], dims=2))
    println("Infidelity " *string(unitary_infidelity(result[:Ũ⃗][:, end], operator_to_iso_vec(target))))
    rollout_states = unitary_rollout(result, system; integrator=exp)
    ΔUs = map(norm, eachslice(rollout_states .- result[:Ũ⃗], dims=2))
    println("Rollout Error "*string(maximum(ΔUs)))
    println("Infidelity "*string(unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target))))
    push!(initialized_infidelities,unitary_infidelity(rollout_states[:, end], operator_to_iso_vec(target)))

end

Angle 0.0
-----------------------
    building dynamics from integrators...
        constructing knot point dynamics functions...
        determining dynamics derivative structure...
            computing jacobian sparsity...
            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:    81906
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     5408
                     variables with only lower bou

  76  1.2245673e-07 5.92e-07 3.65e+01  -4.1 6.57e-03    -  1.00e+00 2.50e-01h  3
  77  2.3721235e-03 4.07e-08 5.00e+01  -4.1 6.77e-03    -  1.00e+00 1.00e+00H  1
  78  1.3795867e-03 4.35e-07 4.11e+01  -4.0 1.64e-03    -  1.00e+00 1.00e+00h  1
  79  4.1859610e-04 1.80e-06 3.48e+00  -4.1 1.16e-02    -  1.00e+00 5.00e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  2.2644017e-04 2.40e-06 4.52e+01  -4.1 7.38e-02    -  1.00e+00 1.25e-01h  4
  81  2.0984938e-04 2.51e-06 5.06e+00  -4.1 2.46e-01    -  4.29e-01 8.01e-03h  7
  82  4.8869128e-05 6.08e-06 1.89e+01  -4.1 2.09e-02    -  1.00e+00 5.00e-01h  2
  83  6.2709566e-05 2.60e-05 2.31e+01  -4.1 1.67e-02    -  1.00e+00 1.00e+00h  1
  84  4.3024922e-05 2.43e-05 2.67e+01  -4.1 9.85e-03    -  1.00e+00 6.25e-02h  5
  85  5.2402153e-05 4.72e-06 2.53e+01  -4.1 6.69e-03    -  1.00e+00 1.00e+00h  1
  86  1.8904110e-06 1.81e-06 2.59e+01  -4.1 6.53e-03    -  1.00e+00 1.00e+00h  1
  87  1.2646820e-04 1.26e-06

  37  6.0112508e-07 2.13e-07 2.73e+01  -4.1 6.67e-02    -  1.00e+00 1.56e-02h  7
  38  5.6445993e-07 2.01e-07 2.29e+01  -4.1 2.40e-03  -2.0 1.00e+00 6.25e-02h  5
  39  3.3352537e-03 2.90e-09 1.05e-02  -4.1 4.62e-03    -  1.00e+00 1.00e+00H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40  1.9849364e-03 1.30e-07 1.95e+01  -4.0 1.63e-03    -  1.00e+00 1.00e+00f  1
  41  2.6400302e-04 6.47e-07 3.29e+01  -4.1 2.92e-02    -  1.00e+00 1.25e-01f  4
  42  7.1974034e-04 3.28e-08 2.89e+01  -4.1 9.09e-04    -  1.00e+00 1.00e+00h  1
  43  2.1405996e-04 8.60e-08 2.54e+01  -4.1 1.46e-03    -  1.00e+00 1.00e+00h  1
  44  4.1708472e-05 1.09e-07 2.46e+01  -4.1 2.56e-03    -  1.00e+00 5.00e-01h  2
  45  8.8563471e-05 4.23e-10 1.73e-03  -4.1 6.95e-05    -  1.00e+00 1.00e+00h  1
  46  2.8456705e-05 9.19e-09 2.36e+01  -4.0 4.78e-04    -  1.00e+00 1.00e+00h  1
  47  9.1917924e-03 1.65e-09 1.98e-02  -4.1 1.93e-02    -  1.00e+00 1.00e+00H  1
  48  1.3009050e-06 1.03e-07

   1  9.5650972e-08 5.36e-11 7.63e-05  -4.0 5.32e-05    -  9.90e-01 1.00e+00h  1
   2  7.8408965e-09 1.06e-09 1.60e-04  -2.6 3.35e-04    -  1.00e+00 2.50e-01h  3
   3  3.6924956e-07 1.13e-09 4.37e-04  -3.9 1.40e-03    -  1.00e+00 1.25e-01h  4
   4  9.8312218e-08 8.84e-10 2.34e-04  -5.9 1.40e-04    -  1.00e+00 1.00e+00h  1
   5  1.9481796e-07 8.18e-10 3.16e-04  -5.9 2.66e-04    -  1.00e+00 2.50e-01h  3
   6  3.1997273e-08 6.88e-10 3.22e-04  -5.9 2.77e-04    -  1.00e+00 2.50e-01h  3
   7  2.7714383e-08 1.93e-10 1.69e-04  -5.9 1.03e-04    -  1.00e+00 1.00e+00h  1
   8  2.3599654e-07 2.23e-10 6.90e-05  -5.9 9.72e-05    -  1.00e+00 1.00e+00h  1
   9  5.1324191e-08 1.86e-10 6.38e-05  -5.9 7.96e-05    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.3541778e-08 2.31e-10 1.01e-04  -5.9 1.35e-04    -  1.00e+00 5.00e-01h  2
  11  1.0654291e-06 1.97e-14 2.50e-04  -4.0 9.88e-05    -  1.00e+00 1.00e+00H  1
  12  6.2954796e-07 4.59e-11

  94  4.2998529e-10 1.44e-15 3.27e-07  -9.0 2.02e-07    -  1.00e+00 1.00e+00h  1
  95  4.2956886e-10 7.77e-16 2.54e-07  -9.0 1.33e-07    -  1.00e+00 1.00e+00h  1
  96  4.2951290e-10 4.33e-15 1.11e-07  -9.0 4.57e-07    -  1.00e+00 1.00e+00H  1
  97  4.2894521e-10 4.44e-16 3.53e-07  -9.0 7.36e-07    -  1.00e+00 1.00e+00H  1
  98  4.2928024e-10 7.77e-16 3.98e-07  -9.0 1.21e-07   0.5 1.00e+00 2.50e-01h  3
  99  4.2955217e-10 1.11e-16 1.96e-07  -9.0 3.98e-07    -  1.00e+00 1.00e+00H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  4.2844549e-10 1.11e-16 5.67e-08  -9.0 1.23e-07    -  1.00e+00 1.00e+00H  1

Number of Iterations....: 100

                                   (scaled)                 (unscaled)
Objective...............:   4.2844548679718647e-10    4.2844548679718647e-10
Dual infeasibility......:   5.6714067819892548e-08    5.6714067819892548e-08
Constraint violation....:   1.1102230246251565e-16    1.1102230246251565e-16
Variable bound vio

...

  58  5.6302339e-09 5.16e-12 1.56e-05  -8.8 1.24e-05    -  1.00e+00 1.00e+00h  1
  59  2.4927880e-09 3.26e-12 1.64e-05  -8.8 6.89e-06    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  60  4.1708073e-09 1.11e-12 1.34e-05  -8.8 5.58e-06    -  1.00e+00 1.00e+00h  1
  61  1.5879470e-09 1.61e-12 4.00e-06  -8.8 1.54e-05    -  1.00e+00 1.00e+00H  1
  62  1.7920863e-09 1.36e-12 7.53e-06  -8.8 6.40e-06    -  1.00e+00 1.00e+00h  1
  63  1.8205857e-09 2.58e-13 6.13e-06  -8.8 2.55e-06    -  1.00e+00 1.00e+00h  1
  64  1.5926590e-09 9.96e-13 3.25e-06  -8.8 5.22e-06    -  1.00e+00 1.00e+00h  1
  65  1.2998992e-09 2.90e-12 8.44e-06  -8.8 9.44e-06    -  1.00e+00 1.00e+00h  1
  66  1.2425858e-09 1.37e-12 4.89e-06  -8.8 6.46e-06    -  1.00e+00 1.00e+00h  1
  67  1.1871063e-09 3.18e-12 8.97e-06  -8.8 2.37e-04    -  1.00e+00 3.12e-02h  6
  68  6.6188777e-09 2.22e-16 1.39e-05  -8.8 8.97e-06    -  1.00e+00 1.00e+00H  1
  69  5.1570556e-09 1.76e-13

  19  3.8813530e-09 1.42e-10 1.03e-04  -8.8 2.56e-04    -  1.00e+00 1.25e-01h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  3.2856144e-09 1.43e-10 1.00e-04  -8.8 3.85e-04    -  1.00e+00 6.25e-02h  5
  21  8.4051110e-07 1.44e-14 2.49e-04  -8.8 1.00e-04    -  1.00e+00 1.00e+00H  1
  22  5.0189048e-07 4.00e-11 1.99e-04  -8.8 3.40e-05    -  1.00e+00 1.00e+00h  1
  23  6.4459494e-11 3.67e-12 2.59e-06  -8.8 1.34e-04    -  1.00e+00 1.00e+00H  1
  24  2.5561126e-10 4.89e-11 4.22e-05  -4.0 2.86e-05    -  1.00e+00 1.00e+00h  1
  25  2.7465614e-10 6.05e-10 1.36e-05  -4.1 5.72e-05    -  1.00e+00 1.00e+00h  1
  26  4.3683762e-08 9.57e-11 5.50e-05  -6.1 6.04e-05    -  1.00e+00 1.00e+00h  1
  27  1.8287991e-08 2.96e-10 9.47e-05  -6.1 2.97e-03    -  1.00e+00 3.12e-02h  6
  28  1.6184768e-08 2.56e-10 8.76e-05  -6.1 3.53e-04    -  1.00e+00 1.25e-01h  4
  29  1.5152128e-08 2.20e-10 8.27e-05  -6.1 2.72e-04    -  1.00e+00 1.25e-01h  4
iter    objective    inf_pr 

            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:    81906
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     5408
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1501
                     variables with only upper bounds:        0
Total number of equality constraints.................:     4600
Total number of ineq

  14  2.5042027e-09 2.63e-12 1.33e-05  -8.8 9.36e-06    -  1.00e+00 1.00e+00h  1
  15  1.4292027e-10 2.57e-12 9.48e-06  -8.8 9.09e-06    -  1.00e+00 1.00e+00h  1
  16  4.3263188e-10 2.58e-12 1.06e-05  -8.8 6.66e-05    -  1.00e+00 3.12e-02h  6
  17  4.1261095e-10 1.54e-12 9.21e-06  -8.8 5.12e-06    -  1.00e+00 5.00e-01h  2
  18  2.0460687e-09 1.32e-12 1.34e-05  -8.8 6.93e-06    -  1.00e+00 1.00e+00h  1
  19  8.4025266e-10 1.45e-12 1.15e-05  -8.8 3.80e-02  -4.0 1.00e+00 1.22e-04h 14
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  2.6987701e-10 1.17e-12 6.51e-06  -8.8 8.18e-06    -  1.00e+00 5.00e-01h  2
  21  9.9086674e-11 1.31e-12 8.87e-06  -8.8 2.66e-04    -  1.00e+00 7.81e-03h  8
  22  1.9547937e-10 1.12e-12 7.58e-06  -8.8 7.30e-06    -  1.00e+00 5.00e-01h  2
  23  6.1980673e-11 1.31e-12 9.03e-06  -8.8 2.23e-05    -  1.00e+00 1.25e-01h  4
  24  8.1128065e-11 1.17e-12 8.30e-06  -8.8 8.72e-06    -  1.00e+00 2.50e-01h  3
  25  5.2798692e-09 1.11e-16

   1  2.1497924e-11 5.88e-15 7.25e-07  -4.0 3.48e-07    -  9.90e-01 1.00e+00h  1
   2  9.1502293e-10 6.51e-10 3.47e-04  -2.6 3.48e-04    -  1.00e+00 5.00e-01f  2
   3  1.7604609e-08 1.37e-11 3.69e-05  -3.9 1.93e-05    -  1.00e+00 1.00e+00h  1
   4  9.6442358e-09 1.95e-11 1.34e-05  -5.9 2.05e-05    -  1.00e+00 1.00e+00h  1
   5  1.4187804e-09 2.28e-11 3.17e-05  -8.8 2.95e-03    -  9.47e-01 3.91e-03h  9
   6  2.0421897e-08 4.85e-11 2.15e-05  -8.8 3.72e-05    -  1.00e+00 1.00e+00h  1
   7  1.5910708e-10 5.12e-11 4.16e-05  -8.8 3.71e-05    -  1.00e+00 1.00e+00h  1
   8  9.1057821e-09 3.07e-11 2.11e-05  -8.8 2.75e-05    -  1.00e+00 5.00e-01h  2
   9  1.4068683e-09 3.24e-11 3.71e-05  -8.8 1.11e-04    -  1.00e+00 1.25e-01h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.4685419e-09 1.67e-11 1.37e-05  -8.8 2.67e-05    -  1.00e+00 1.00e+00h  1
  11  2.9347446e-08 1.65e-11 3.80e-05  -8.8 1.37e-05    -  1.00e+00 1.00e+00h  1
  12  1.9660599e-08 3.93e-12

In [9]:
initialized_infidelities

21-element Vector{Any}:
 1.1374029382893447e-5
 0.012312301542026671
 0.04894348372552204
 0.10899347541379933
 0.19098307098567313
 0.292893216909881
 0.41221471882833927
 0.5460081502831464
 0.690982973692575
 0.8435968658379659
 0.9999997596497144
 0.8435655582734706
 0.6909831757942979
 0.5460095007061931
 0.41221474563935556
 0.29289312340540163
 0.19098356528683336
 0.10899352159851183
 0.04894346390212734
 0.012311661213206149
 8.43769498715119e-15